In [1]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import geopandas as gpd
from descartes import PolygonPatch

In [2]:
df = pd.read_csv("data/full_run_2.zip",skiprows=6)

In [3]:
#remove all the columns that have the same value in all rows.
for c in df.columns:
    if len(df[c].unique()) < 2:
        df = df.drop(columns=c)

In [4]:
df

,[run number],transform-houses?,increase-social-housing?,build-more-houses?,[step],KPI-available_buy_houses,KPI-available_part_rent_houses,KPI-available_social_rent_houses,KPI-avg_income,KPI-avg_price,...,KPI-p-sc-upper,KPI-p-dutch,KPI-p-other-western,KPI-p-antilles,KPI-p-morocco,KPI-p-suriname,KPI-p-turkey,KPI-p-indonesian,KPI-p-eastern-eu,KPI-p-other-nonwestern
0,21,False,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,False,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14,False,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,29,False,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,23,False,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,251,True,True,True,39,[[40 0] [61 2327] [17 0] [53 1] [68 332] [48 4...,[[40 0] [61 629] [17 0] [53 0] [68 44] [48 436...,[[40 0] [61 4312] [17 0] [53 0] [68 14] [48 18...,[[40 23331] [61 32312] [17 21011] [53 40040] [...,[[40 218070] [61 302248] [17 197135] [53 37197...,...,[[40 0] [61 0.001] [17 0] [53 0.048] [68 0.74]...,[[40 0.978] [61 0.455] [17 0.004] [53 0.616] [...,[[40 0.003] [61 0.086] [17 0.002] [53 0.107] [...,[[40 0.001] [61 0.046] [17 0.004] [53 0.016] [...,[[40 0.002] [61 0.053] [17 0.939] [53 0.02] [6...,[[40 0.002] [61 0.05] [17 0] [53 0.04] [68 0.0...,[[40 0.004] [61 0.067] [17 0.011] [53 0.055] [...,[[40 0.002] [61 0.019] [17 0.019] [53 0.022] [...,[[40 0.003] [61 0.051] [17 0.01] [53 0.033] [6...,[[40 0.005] [61 0.173] [17 0.011] [53 0.091] [...
10492,250,True,True,True,40,[[113 1573] [91 436] [18 0] [40 1] [2 1221] [6...,[[113 402] [91 301] [18 6] [40 1] [2 1] [66 10...,[[113 883] [91 156] [18 0] [40 1] [2 0] [66 21...,[[113 39586] [91 55522] [18 20179] [40 23384] ...,[[113 370597] [91 518308] [18 189216] [40 2184...,...,[[113 0.156] [91 0.532] [18 0] [40 0] [2 0.006...,[[113 0.591] [91 0.764] [18 0.031] [40 0.965] ...,[[113 0.061] [91 0.122] [18 0.627] [40 0.004] ...,[[113 0.012] [91 0.004] [18 0.252] [40 0.002] ...,[[113 0.031] [91 0] [18 0.008] [40 0.002] [2 0...,[[113 0.035] [91 0.055] [18 0.002] [40 0.002] ...,[[113 0.099] [91 0.02] [18 0.011] [40 0.011] [...,[[113 0.035] [91 0.004] [18 0.014] [40 0.005] ...,[[113 0.028] [91 0.004] [18 0.027] [40 0.003] ...,[[113 0.108] [91 0.027] [18 0.028] [40 0.006] ...
10493,248,True,True,True,40,[[74 321] [71 14096] [114 947] [30 3617] [93 1...,[[74 72] [71 5465] [114 473] [30 2094] [93 1] ...,[[74 8] [71 2877] [114 591] [30 1079] [93 58] ...,[[74 89739] [71 49057] [114 37292] [30 26129] ...,[[74 839172] [71 457935] [114 348091] [30 2567...,...,[[74 0.859] [71 0.547] [114 0.048] [30 0] [93 ...,[[74 0.606] [71 0.62] [114 0.49] [30 0.442] [9...,[[74 0.196] [71 0.167] [114 0.054] [30 0.102] ...,[[74 0.008] [71 0.018] [114 0.032] [30 0.013] ...,[[74 0.003] [71 0.007] [114 0.04] [30 0.028] [...,[[74 0.068] [71 0.045] [114 0.046] [30 0.024] ...,[[74 0.008] [71 0.021] [114 0.135] [30 0.121] ...,[[74 0.005] [71 0.008] [114 0.049] [30 0.102] ...,[[74 0.022] [71 0.04] [114 0.026] [30 0.118] [...,[[74 0.084] [71 0.074] [114 0.128] [30 0.05] [...
10494,256,True,True,True,40,[[1 1546] [65 3865] [110 0] [92 4738] [74 355]...,[[1 754] [65 2969] [110 0] [92 1524] [74 113] ...,[[1 1315] [65 224] [110 3] [92 1523] [74 8] [7...,[[1 32577] [65 54818] [110 60302] [92 39148] [...,[[1 305988] [65 512960] [110 541842] [92 36839...,...,[[1 0.005] [65 0.711] [110 0] [92 0.205] [74 0...,[[1 0.757] [65 0.524] [110 0] [92 0.714] [74 0...,[[1 0.084] [65 0.195] [110 0] [92 0.114] [74 0...,[[1 0.011] [65 0.007] [110 0] [92 0.004] [74 0...,[[1 0] [65 0.004] [110 0] [92 0.001] [74 0] [7...,[[1 0.013] [65 0.044] [110 0] [92 0.069] [74 0...,[[1 0.027] [65 0.015] [110 0] [92 0.019] [74 0...,[[1 0.005] [65 0.011] [110 0] [92 0.011] [74 0...,[[1 0.035] [65 0.039] [110 0] [92 0.026] [74 0...,[[1 0.068] [65 0.161] [110 1] [92 0.042] [74 0...
